# 감성 사전을 이용한 영화 리뷰 감성 분석

**NLTK 영화 리뷰 데이터 준비**

In [ ]:
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews

In [2]:
print('review count:', len(movie_reviews.fileids()))    # 영화 리뷰 문서의 id를 반환
print('samples of file ids:', movie_reviews.fileids()[:10])     # id를 10개까지만 출력
print('categories of reviews:', movie_reviews.categories())     # label, 즉 긍정인지 부정인지에 대한 분류
print('num of "neg" reviews:', len(movie_reviews.fileids(categories='neg')))    # label이 부정인 문서들의 id를 반환
print('num of "pos" reviews:', len(movie_reviews.fileids(categories='pos')))    # label이 긍정인 문서들의 id를 반환

review count: 2000
samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']
categories of reviews: ['neg', 'pos']
num of "neg" reviews: 1000
num of "pos" reviews: 1000


In [3]:
fileid = movie_reviews.fileids()[0]     # 첫번째 문서의 id를 반환

print('id of the first review:', fileid)
print('part of the first review:', movie_reviews.raw(fileid)[:500])     # 첫번째 문서의 내용을 500자까지만 출력
print('sentiment of the first review:', movie_reviews.categories(fileid))   # 첫번째 문서의 감성

id of the first review: neg/cv000_29416.txt
part of the first review: plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt
sentiment of the first review: ['neg']


In [4]:
fileids = movie_reviews.fileids()       # movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids]         # file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

**TextBlob을 이용한 감성 분석**

In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora

In [6]:
from textblob import TextBlob

result = TextBlob(reviews[0])
print(result.sentiment)

Sentiment(polarity=0.06479782948532947, subjectivity=0.5188408350908352)


In [7]:
# textblob 극성의 범위 [-1.0(부정), 1.0(긍정)], 주관성의 범위 [0.0(객관적), 1.0(주관적)]

def sentiment_TextBlob(docs):
    results = []

    for doc in docs:
        testimonial = TextBlob(doc)
        if testimonial.sentiment.polarity > 0:
            results.append('pos')
        else:
            results.append('neg')
    return results

In [8]:
from sklearn.metrics import accuracy_score

print('TextBlob을 이용한 리뷰 감성분석의 정확도:', accuracy_score(categories, sentiment_TextBlob(reviews)))

TextBlob을 이용한 리뷰 감성분석의 정확도: 0.6


**AFINN을 이용한 감성 분석**

In [ ]:
!pip install afinn

In [28]:
from afinn import Afinn

result = Afinn(emoticons=True)
print(result.score(reviews[0]))

-16.0


In [10]:
def sentiment_Afinn(docs):
    afn = Afinn(emoticons=True)
    results = []

    for doc in docs:
        if afn.score(doc) > 0:
            results.append('pos')
        else:
            results.append('neg')
    return results

In [11]:
print('Afinn을 이용한 리뷰 감성분석의 정확도:', accuracy_score(categories, sentiment_Afinn(reviews)))

Afinn을 이용한 리뷰 감성분석의 정확도: 0.664


**VADER를 이용한 감성 분석**

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [34]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

result = SentimentIntensityAnalyzer()
print(result.polarity_scores(reviews[0]))

{'neg': 0.093, 'neu': 0.762, 'pos': 0.145, 'compound': 0.9924}


In [30]:
# compound의 범위 [-1.0(부정), 1.0(긍정)], compound <= -0.05: 부정 / -0.05 < compound < 0.05: 중립 / compound >= 0.05: 글정 (일반적인 임계값으로 문서마다 다를 수 있음)
# neg, neu, pos의 태그는 분석할 텍스트 내의 각 태그에 속하는 텍스트의 비율

def sentiment_vader(docs):
    analyser = SentimentIntensityAnalyzer()
    results = []

    for doc in docs:
        score = analyser.polarity_scores(doc)
        if score['compound'] > 0:
            results.append('pos')
        else:
            results.append('neg')

    return results

In [31]:
print('Vader을 이용한 리뷰 감성분석의 정확도:', accuracy_score(categories, sentiment_vader(reviews)))

Vader을 이용한 리뷰 감성분석의 정확도: 0.635
